To set up and begin running DeepGlycanSite, we first need to create the environment that contains all of its dependencies. I have already established the environment, we just need to run it and make sure its working. 

In [ ]:
from pathlib import Path
import subprocess

import docker
import docker.errors
from docker.models.containers import Container

root = Path().resolve() #Root of this local project directory   

try:
    client: docker.client.DockerClient = docker.from_env()
except Exception as e:
    print(e)

client

In [ ]:
# Run this in command line, not in Jupyter Notebook.

!docker compose up --build deepglycansite

In [ ]:
project_name = "deepglycansite_network" #Do Not Change
service_name = "deepglycansite" #Here, choose the docking service you want to use as named in 'docker-compose.yml'

containers = client.containers.list(all=True)

app_container: Container = None

for container in containers:
    if f"{service_name}" in container.name:
        print(f"Starting container: {container.name}")
        try:
            container.start()
            print(container.name)
            app_container = container
            break
        except docker.errors.APIError as e:
            print(e)
    

In [ ]:
in_folder: str = "janique_run" #Change to your name
ligand_sdf = f"shared_data/{in_folder}/gangliotriose.sdf" #Change /folder/example.sdf to the place where your file is in this directory.
print(ligand_sdf)
receptor_pdb = f"shared_data/{in_folder}/CBM_A.pdb" #Change /folder/receptor.pdb the same way
print(receptor_pdb)
# Ensure the input folder exists in the docker container

output_file_name = "CBM-A_w_gangliotriose.txt"

cmd = f"mkdir -p {in_folder}"

exit_code, output = app_container.exec_run(cmd)

app_container.exec_run(f"mkdir -p User")

example_path = Path(str(root) + "/binding_site_files/IQZ2_sites.txt").resolve()
exit_code, output

In [ ]:
cmd = f"python3 single_case_prediction.py --conf P2Y14_example/hparams.yaml --ckpt_path ckpts/with_ligand.ckpt --input_fn {receptor_pdb},{ligand_sdf} --out_path User/ --output_fn {output_file_name}"

In [ ]:
app_container.exec_run(
    cmd
)